# 设计文档

## 文档说明

本文档包含多码率LDPC码的译码器的主要设计思路，以及实现细节，对于设计以及代码中的实现内的 trick 会着重提及，以保证该项目的可用性。

## 整体介绍

## 主要组成部分

这是一个经典的 QC-LDPC 码硬件译码器的实现，基于 cpp 的译码算法可以参考另一个仓库中的内容。在这个经典的LDPC码硬件译码器的实现中，我们会设计到如下部分:

计算模块：
+ CPU
+ VPU

控制模块：
+ controller

存储模块：
+ llr_ram








## CPU




In [ ]:
# 地址生成
base = "\n\
always @(posedge clk) begin\n\
	if (vpu_addr_ena == 1'b1)  begin\n\
		vpu_addr_{port} <= vpu_addr_{port} + 1'b1;\n\
	end\n\
	else\n\
		vpu_addr_{port} <= row_start_{port};\n\
end\
"

# for x in range(24):
# 	print(base.format(port=x))


base_cpu = "\n\
always @(posedge clk) begin\n\
	if (cpu_addr_ena == 1'b1)  begin\n\
		cpu_addr_{port} <= cpu_addr_{port} + 1'b1;\n\
	end\n\
	else\n\
		cpu_addr_{port} <= 0;\n\
end\
"

for x in range(24):
	print(base_cpu.format(port=x))

## VPU


## controller


## llr_ram

## do 文件模板

```do

# 创建库及映射库
vlib work
vmap work work

# 这里参数都是可选的 不加其实也可以
# vlog -incr -work work ../../ip/

vlog -incr -work work ../../rtl/ldpc_vpu.v
vlog -incr -work work ../../tb/tb_ldpc_vpu.v

# 固定格式，虽不知为何一定要用 -voptargs=+acc 但默认就是这种方式
# 不加的话代码跑不下去
vsim -voptargs=+acc work.tb_ldpc_vpu

# 添加波形
# 这里可以引入更加复杂的设置
add wave -radix decimal sim:/tb_ldpc_vpu/*
# add wave sim:/tb_ldpc_vpu/inst_ldpc_vpu/*

# 指定以何种进制显示
# Use binary, octal, decimal, signed, unsigned, hexadecimal, ascii or default. 默认二进制
# add wave -radix hex sim:/tb_ldpc_vpu/llr_all
# 指定以何种颜色显示
# add wave -color Yellow sim:/tb_ldpc_vpu/llr_out_0

# 选择需要加入的文件
do wave.do

view wave

run -all

```

## 多码率设计

多余多码率的 CPU 的设计需要更复杂一些，需要考虑到变化和不变的部分。

1. 校验矩阵
2. 生成矩阵

基矩阵的几种情况：

1. $4 \times 8$
2. $4 \times 12$
3. $4 \times 16$
4. $4 \times 20$
5. $4 \times 24$


因此，CPU 的连线会有变化，在需要增加选择器的情况下，我认为继续使用之前那种 `for` 循环生成子模块的方法似乎不太能继续用了，有些含糊不清。



CPU 结构的变化，需要增加更多的选择器。


In [3]:
# comp3in2out dut

base = '''
comp3in2out #(
  .WIDTH(LLR_WIDTH-1)
) inst_comp3in2out_{idx} (
  .in_1     (llr_in_reg[{p1}])  ,
  .in_2     (llr_in_reg[{p2}]),
  .in_3     (llr_in_reg[{p3}]),
  .min_v    (as_min[{idx}])        ,
  .submin_v (as_submin[{idx}])
)'''

for i in range(8):
  print(base.format(idx=i, p1=i*3, p2=i*3+1, p3=i*3+2))




comp3in2out #(
  .WIDTH(LLR_WIDTH-1)
) inst_comp3in2out_0 (
  .in_1     (llr_in_reg[0])  ,
  .in_2     (llr_in_reg[1]),
  .in_3     (llr_in_reg[2]),
  .min_v    (as_min[0])        ,
  .submin_v (as_submin[0])
)

comp3in2out #(
  .WIDTH(LLR_WIDTH-1)
) inst_comp3in2out_1 (
  .in_1     (llr_in_reg[3])  ,
  .in_2     (llr_in_reg[4]),
  .in_3     (llr_in_reg[5]),
  .min_v    (as_min[1])        ,
  .submin_v (as_submin[1])
)

comp3in2out #(
  .WIDTH(LLR_WIDTH-1)
) inst_comp3in2out_2 (
  .in_1     (llr_in_reg[6])  ,
  .in_2     (llr_in_reg[7]),
  .in_3     (llr_in_reg[8]),
  .min_v    (as_min[2])        ,
  .submin_v (as_submin[2])
)

comp3in2out #(
  .WIDTH(LLR_WIDTH-1)
) inst_comp3in2out_3 (
  .in_1     (llr_in_reg[9])  ,
  .in_2     (llr_in_reg[10]),
  .in_3     (llr_in_reg[11]),
  .min_v    (as_min[3])        ,
  .submin_v (as_submin[3])
)

comp3in2out #(
  .WIDTH(LLR_WIDTH-1)
) inst_comp3in2out_4 (
  .in_1     (llr_in_reg[12])  ,
  .in_2     (llr_in_reg[13]),
  .in_3     (llr_in_reg[14]